In [ ]:
# default_exp core

# Core

> API details.

In [ ]:
#export 

class ReconstructionOptions:
    def __init__(self):
        pass

In [ ]:
#export 
from smpr3d.util import fourier_coordinates_2D
from smpr3d.torch_imports import *
import numpy as np 
from numpy.fft import fftshift

class SMeta:
    def __init__(self, take_beams, dx, S_shape, MY, MX, device):
        self.f = S_shape[1:] / np.array([MY, MX])
        self.S_shape = S_shape
        self.q = th.as_tensor(fourier_coordinates_2D(S_shape[1:], dx.numpy(), centered=False), device=device)
        self.qf = th.as_tensor(fourier_coordinates_2D([MY, MX], dx.numpy(), centered=False), device=device)
        self.q2 = th.norm(self.q, dim=0) ** 2
        self.qf2 = th.norm(self.qf, dim=0) ** 2
        self.q_coords = th.from_numpy(
            fftshift(np.array(np.mgrid[-MY // 2:MY // 2, -MX // 2:MX // 2]), (1, 2))).to(device)
        self.r_indices = th.from_numpy(np.mgrid[:MY, :MX]).to(device)
        self.take_beams = take_beams.to(device)
        
        B = th.sum(take_beams).item()
        self.beam_numbers = th.ones_like(take_beams, dtype=th.long, device=device) * -1
        self.beam_numbers[take_beams] = th.arange(0, B, device=device)
        self.q_b = th.stack([self.qf[0,take_beams],self.qf[1,take_beams]]).T
        self.q_b_coords = th.from_numpy(np.mgrid[:MY, :MX]).to(device)
        self.q_dft = th.from_numpy(fourier_coordinates_2D([MY, MX], [1, 1], centered=False)).to(device)
        
        #S_shape          array (3,)        
        #q              (NY, NX) x
        #qf             (MY, MX) x
        #q2             (NY, NX) x
        #q2f            (MY, MX) x
        #f              (2,)
        #q_coords       (MY, MX)
        #r_indices      (NY, NX)
        #take_beams     (MY, MX)
        #beam_numbers   (MY, MX)
        #q_b            (B, 2)
        #q_b_coords     (B, 2)